![jupyter](./banner.png)

chatterbot是一款python接口的，基于一系列规则和机器学习算法完成的聊天机器人。具有结构清晰，可扩展性好，简单实用的特点。

## 安装

是的，安装超级简单，用pip就可以啦  
pip install chatterbot

## 各式各样的Adapter

大家已经知道chatterbot的聊天逻辑和输入输出以及存储，是由各种adapter来限定的，我们先看看流程图，一会软再一起看点例子，看看怎么用。

![jupyter](./chatterbot-process-flow.svg)

## 基础版本

In [11]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer


# Create a new instance of a ChatBot
bot = ChatBot(
    'Example Bot',# 当前创建的Bot名
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        { #Low confidence response例子,chatterbot.logic.LowConfidenceAdapter已经被移除
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'I am sorry, but I do not understand.',
            'maximum_similarity_threshold': 0.90 
        }
    ]
)

trainer = ListTrainer(bot)

# Train the chat bot with a few responses
# 注意这里的语料是对话,有应答
trainer.train([
    'How can I help you?',
    'I want to create a chat bot',
    'Have you read the documentation?',
    'No, I have not',
    'This should help get you started: http://chatterbot.rtfd.org/en/latest/quickstart.html'
])

# Get a response for some unexpected input
question = 'How do I make an omelette?'
print(question)
response = bot.get_response(question)
print(response)

print("\n")
question = 'how to make a chat bot?'
print(question)
response = bot.get_response(question)
print(response)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
List Trainer: [####################] 100%
How do I make an omelette?
I am sorry, but I do not understand.


how to make a chat bot?
I am sorry, but I do not understand.


## 处理时间和数学计算的Adapter

In [14]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot


bot = ChatBot(
    "Math & Time Bot",
    logic_adapters=[
        "chatterbot.logic.MathematicalEvaluation",
        "chatterbot.logic.TimeLogicAdapter"
    ],
    input_adapter="chatterbot.input.VariableInputTypeAdapter",
    output_adapter="chatterbot.output.OutputAdapter"
)

# 进行数学计算
question = "What is 4 + 9?"
print(question)
response = bot.get_response(question)
print(response)

print("\n")

# 回答和时间相关的问题
question = "What time is it?"
print(question)
response = bot.get_response(question)
print(response)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
What is 4 + 9?
4 + 9 = 13


What time is it?
The current time is 05:22 PM


## 导出语料到json文件

In [20]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

'''
如果一个已经训练好的chatbot，你想取出它的语料，用于别的chatbot构建，可以这么做
'''

chatbot = ChatBot('Export Example Bot')

# First, lets train our bot with some data
trainer = ChatterBotCorpusTrainer(chatbot)

trainer.train('chatterbot.corpus.english')

# Now we can export the data to a file
trainer.export_for_training('./my_export.json')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [##############

## 反馈式学习聊天机器人

In [22]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
import logging

"""
反馈式的聊天机器人，会根据你的反馈进行学习
"""

# 把下面这行前的注释去掉，可以把一些信息写入日志中
logging.basicConfig(level=logging.INFO)

# 创建一个聊天机器人
bot = ChatBot(
    'Feedback Learning Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        'chatterbot.logic.BestMatch'
    ],
    input_adapter='chatterbot.input.TerminalAdapter',
    output_adapter='chatterbot.output.TerminalAdapter'
)

DEFAULT_SESSION_ID = bot.default_session.id


def get_feedback():
    from chatterbot.utils import input_function

    text = input_function()

    if 'Yes' in text:
        return True
    elif 'No' in text:
        return False
    else:
        print('Please type either "Yes" or "No"')
        return get_feedback()


print('Type something to begin...')

# 每次用户有输入内容，这个循环就会开始执行
while True:
    try:
        input_statement = bot.input.process_input_statement()
        statement, response = bot.generate_response(input_statement, DEFAULT_SESSION_ID)

        print('\n Is "{}" this a coherent response to "{}"? \n'.format(response, input_statement))

        if get_feedback():
            bot.learn_response(response,input_statement)

        bot.output.process_response(response)

        # 更新chatbot的历史聊天数据
        bot.conversation_sessions.update(
            bot.default_session.id_string,
            (statement, response, )
        )

    # 直到按ctrl-c 或者 ctrl-d 才会退出
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Type something to begin...


AttributeError: 'ChatBot' object has no attribute 'input'

In [32]:
from chatterbot import ChatBot
from chatterbot.conversation import Statement

"""
反馈式机器人
This example shows how to create a chat bot that
will learn responses based on an additional feedback
element from the user.
"""

# Uncomment the following line to enable verbose logging
# import logging
# logging.basicConfig(level=logging.INFO)

# Create a new instance of a ChatBot
bot = ChatBot(
    'Feedback Learning Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter'
)


def get_feedback():

    text = input()

    if 'yes' in text.lower():
        return True
    elif 'no' in text.lower():
        return False
    else:
        print('Please type either "Yes" or "No"')
        return get_feedback()


print('Type something to begin...')

# The following loop will execute each time the user enters input
while True:
    try:
        input_statement = Statement(text=input())
        response = bot.generate_response(
            input_statement
        )

        print('\n Is "{}" a coherent response to "{}"? \n'.format(
            response.text,
            input_statement.text
        ))
        # 更新聊天历史数据
        if get_feedback():
            print('please input the correct one')
            correct_response = Statement(text=input())
            bot.learn_response(correct_response, input_statement)
            print('Responses added to bot!')

    # Press ctrl-c or ctrl-d on the keyboard to exit
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Type something to begin...
hello


INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: Hello 1.0
INFO:chatterbot.chatterbot:Using "Hello" as a close match to "hello" with a confidence of 1.0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "Hi"
INFO:chatterbot.chatterbot:Selecting response from 2 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 2 options.
INFO:chatterbot.chatterbot:Response selected. Using "Greetings!"
INFO:chatterbot.chatterbot:BestMatch selected "Greetings!" as a response with a confidence of 1.0



 Is "Greetings!" a coherent response to "hello"? 

No
how are you


INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: Who are you? 0.87
INFO:chatterbot.chatterbot:Similar text found: How are you? 0.96
INFO:chatterbot.chatterbot:Using "How are you?" as a close match to "how are you" with a confidence of 0.96
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "Hi"
INFO:chatterbot.chatterbot:Selecting response from 4 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 4 options.
INFO:chatterbot.chatterbot:Response selected. Using "I am on the Internet."
INFO:chatterbot.chatterbot:BestMatch selected "I am on the Internet." as a response with a confidence of 0.96



 Is "I am on the Internet." a coherent response to "how are you"? 

No
how old are you


INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Using "how old are you" as a close match to "how old are you" with a confidence of 0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "Hi"
INFO:chatterbot.chatterbot:Selecting response from 1152 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 1152 options.
INFO:chatterbot.chatterbot:Response selected. Using "How can I help you?"
INFO:chatterbot.chatterbot:BestMatch selected "How can I help you?" as a response with a confidence of 0



 Is "How can I help you?" a coherent response to "how old are you"? 

exit()


In [33]:
"""
This example shows how to train a chat bot using the
Ubuntu Corpus of conversation dialog.
"""
import logging
from chatterbot import ChatBot
from chatterbot.trainers import UbuntuCorpusTrainer

# Enable info level logging
logging.basicConfig(level=logging.INFO)

chatbot = ChatBot('Example Bot')

trainer = UbuntuCorpusTrainer(chatbot)

# Start by training our bot with the Ubuntu corpus data
trainer.train()

# Now let's get a response to a greeting
response = chatbot.get_response('How are you doing today?')
print(response)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[                                                  ]

KeyboardInterrupt: 

In [38]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

#手动设置一些语料
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer
 
bot = ChatBot("Training demo")
Chinese_bot = ListTrainer(bot)
Chinese_bot.train([
    '你好',
    '你好',
    '有什么能帮你的？',
    '想买数据科学的课程',
    '具体是数据科学哪块呢？',
    '机器学习',
])
 
# 测试一下
question = '你好'
print(question)
response = bot.get_response(question)
print(response)

print("\n")

question = '请问哪里能买数据科学的课程'
print(question)
response = bot.get_response(question)
print(response)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
List Trainer: [####################] 100%

INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: 你好 1.0
INFO:chatterbot.chatterbot:Using "你好" as a close match to "你好" with a confidence of 1.0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "Hi"
INFO:chatterbot.chatterbot:Selecting response from 6 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 6 options.
INFO:chatterbot.chatterbot:Response selected. Using "你好"
INFO:chatterbot.chatterbot:BestMatch selected "你好" as a response with a confidence of 1.0



你好


INFO:chatterbot.chatterbot:Adding "你好" as a response to "请问哪里能买数据科学的课程"
INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: 请问哪里能买数据科学的课程 1.0
INFO:chatterbot.chatterbot:Using "请问哪里能买数据科学的课程" as a close match to "请问哪里能买数据科学的课程" with a confidence of 1.0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "你好"
INFO:chatterbot.chatterbot:1. Excluding recent repeated response of "Hi"
INFO:chatterbot.chatterbot:No responses found. Generating alternate response list.
INFO:chatterbot.chatterbot:No known response to the input was found. Selecting a random response.
INFO:chatterbot.chatterbot:BestMatch selected "i could always improve myself compared to the pack." as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "请问哪里能买数据科学的课程" as a response to "你好"


你好


请问哪里能买数据科学的课程
i could always improve myself compared to the pack.


## 利用已经提供好的小中文语料库

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
 
chatbot = ChatBot("ChineseChatBot")
ChatterBot = ChatterBotCorpusTrainer(chatbot)
 
# 使用中文语料库训练它
ChatterBot.train("chatterbot.corpus.chinese")
 
# 开始对话
while True:
    print(chatbot.get_response(input(">")))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [##################

INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: 你好 1.0
INFO:chatterbot.chatterbot:Using "你好" as a close match to "你好" with a confidence of 1.0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "你好"
INFO:chatterbot.chatterbot:Selecting response from 6 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 6 options.
INFO:chatterbot.chatterbot:Response selected. Using "有什么能帮你的？"
INFO:chatterbot.chatterbot:BestMatch selected "有什么能帮你的？" as a response with a confidence of 1.0
INFO:chatterbot.chatterbot:Adding "你好" as a response to "请问哪里能买数据科学的课程"


有什么能帮你的？
>我想买本书


INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Using "我想买本书" as a close match to "我想买本书" with a confidence of 0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "你好"
INFO:chatterbot.chatterbot:No responses found. Generating alternate response list.
INFO:chatterbot.chatterbot:No known response to the input was found. Selecting a random response.
INFO:chatterbot.chatterbot:BestMatch selected "别担心" as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "我想买本书" as a response to "你好"


别担心
>有吗


INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Using "有吗" as a close match to "有吗" with a confidence of 0
INFO:chatterbot.chatterbot:0. Excluding recent repeated response of "你好"
INFO:chatterbot.chatterbot:No responses found. Generating alternate response list.
INFO:chatterbot.chatterbot:No known response to the input was found. Selecting a random response.
INFO:chatterbot.chatterbot:BestMatch selected "That isn't my name, but it is a useful way to refer to me." as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "有吗" as a response to "我想买本书"


That isn't my name, but it is a useful way to refer to me.
